In [4]:
import os
import csv
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from PIL import Image
import pickle
from sklearn.utils import resample
from matplotlib.pyplot import bar
from skimage.exposure import histogram
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [37]:

# Show the figures / plots inside the notebook
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 


def showHist(img):
    # An "interface" to matplotlib.axes.Axes.hist() method
    plt.figure()
    imgHist = histogram(img, nbins=256)
    
    bar(imgHist[1].astype(np.uint8), imgHist[0], width=0.8, align='center')


In [27]:
def resize_image(img): 
    # Resize image to target size
    # Get original dimensions
    original_height, original_width = img.shape[:2]
    target_width, target_height = 32 , 32
# 
    # Calculate scaling factor while maintaining aspect ratio
    scale = min(target_width / original_width, target_height / original_height)

    # Calculate new dimensions after scaling
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Calculate padding to center the resized image
    pad_top = (target_height - new_height) // 2
    pad_bottom = target_height - new_height - pad_top
    pad_left = (target_width - new_width) // 2
    pad_right = target_width - new_width - pad_left

    # Pad the resized image with white (255)
    padded_img = cv2.copyMakeBorder(
        resized_img,
        pad_top, pad_bottom, pad_left, pad_right,
        borderType=cv2.BORDER_CONSTANT,
        value=[255, 255, 255]  # White background
    )
    
    # padded_img[padded_img<200]=0
    # padded_img[padded_img>=200]=255
    # show_images([padded_img],["padded imag"])

    return padded_img
    

In [26]:
def rename_and_convert_images(folder_path):
    # Get all files in the folder
    files = os.listdir(folder_path)
    
    # Filter image files and sort them
    image_files = [f for f in files if f.lower().endswith(('.png', '.bmp', '.jpg', '.jpeg'))]
    image_files.sort()  # Optional: Ensures renaming follows sorted order

    # Process each image
    for idx, file_name in enumerate(image_files, start=1):
        old_path = os.path.join(folder_path, file_name)
        
        # Set new name with index and PNG extension
        new_name = f"{idx}.png"
        new_path = os.path.join(folder_path, new_name)
        
        try:
            # Convert BMP to PNG and resize
            img = cv2.imread(old_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Failed to load image: {file_name}")
                continue
            
            # background_pixel = img[0, 0]
            # if background_pixel < 128:  # Background is not white (dark)
            #     # print(f"Inverting background for {file_name}")
            #     img = cv2.bitwise_not(img)  # Invert the colors

            # Resize to 32x32
            resized_img = resize_image(img)
            
            # Save the resized image using PIL for PNG format
            pil_img = Image.fromarray(resized_img)
            pil_img.save(new_path, "PNG")
            
            # Remove the old BMP file
            os.remove(old_path)
        
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
            continue  # Skip to the next file if an error occurs

    print("Renaming, resizing, and conversion completed.")



In [7]:
def extract_hog_features(img):
    # Ensure input is a NumPy array
    if img is None or not isinstance(img, np.ndarray):
        raise ValueError("Invalid input image. Ensure it's loaded correctly.")
   
    # Define HOG descriptor parameters
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    block_size = (block_size_in_cells[1] * cell_size[1],
                  block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9
    
    # Initialize HOG descriptor
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    
    # Compute HOG features
    h = hog.compute(img)
    return h.flatten()


In [8]:

def extract_keypoints_and_descriptors(img):
    """
    Extract SIFT keypoints and descriptors from an image.
    """
    if img is None or not isinstance(img, np.ndarray):
        raise ValueError("Invalid input image. Ensure it's loaded correctly.")
    

    # Create a SIFT detector and extract descriptors
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(padded_img, None)

    if descriptors is None:
        descriptors = np.array([])  # Handle empty descriptors

    return keypoints, descriptors

def compute_bow_histogram(descriptors, kmeans, k):
    """
    Compute a Bag of Words histogram for a single image.
    """
    histogram = np.zeros(k)
    if descriptors.size > 0:  # Ensure descriptors are not empty
        cluster_assignments = kmeans.predict(descriptors)
        for cluster in cluster_assignments:
            histogram[cluster] += 1
    return histogram

def process_images_to_sift_bow(label_folder, output_csv, k=100):
    """
    Process images in a folder, extract SIFT Bag of Words features,
    and save the features into a CSV file.
    """
    all_descriptors = []

    # Step 1: Extract SIFT descriptors from all images
    for file_name in os.listdir(label_folder):
        file_path = os.path.join(label_folder, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to load image: {file_name}")
            continue

        _, descriptors = extract_keypoints_and_descriptors(img)
        if descriptors.size > 0:
            all_descriptors.append(descriptors)

    if len(all_descriptors) == 0:
        print("No descriptors extracted from images.")
        return

    all_descriptors = np.vstack(all_descriptors)  # Combine all descriptors

    # Step 2: Perform K-Means clustering to create visual words
    kmeans = KMeans(n_clusters=k, random_state=0).fit(all_descriptors)

    # Step 3: Compute BoW histograms for each image
    features = []
    for file_name in os.listdir(label_folder):
        file_path = os.path.join(label_folder, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        _, descriptors = extract_keypoints_and_descriptors(img)
        histogram = compute_bow_histogram(descriptors, kmeans, k)
        features.append(histogram)

    # Step 4: Save features to a CSV file
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f"Feature_{i+1}" for i in range(k)] + ["Label"])  # Add header
        for histogram in features:
            writer.writerow(list(histogram))  # Save histogram

    print(f"SIFT Bag of Words features saved to {output_csv}")


In [9]:
def process_images_in_folder(label_folder, label, output_csv="features_with_labels.csv"):
    # Check if the CSV file exists
    file_exists = os.path.isfile(output_csv)
    
    with open(output_csv, "a", newline="") as file:
        writer = csv.writer(file)
        
        # Write header if the file doesn't exist
        if not file_exists:
            num_features = 1764  # Adjust to match your HOG settings
            header = [f"x{i+1}" for i in range(num_features)] + ["y"]
            writer.writerow(header)
            print(f"Created {output_csv} and added header.")
        
        # Iterate through all image files in the label folder
        for file_name in os.listdir(label_folder):
            file_path = os.path.join(label_folder, file_name)
            
            # # Skip non-image files
            # if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            #     continue
            
            # Load the image
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Use grayscale for HOG
            if img is None:
                print(f"Failed to load image: {file_name}")
                continue
            
            # Extract HOG features
            features = extract_hog_features(img)
            # print(features)
            # Append the label and write to the CSV
            writer.writerow(np.append(features, label))
            print(f"Processed: {file_name} with label {label}")

    print(f"Features and labels saved to {output_csv}")


In [ ]:
# # Provide the folder path containing the images
folder_path = "./dataset/b_16"
rename_and_convert_images(folder_path)

In [ ]:

# label_folder = "./dataset/a_1"
# process_images_in_folder(label_folder,"11","./csvs/a_1_features_with_labels_11.csv")

# label_folder = "./dataset/a_2"
# process_images_in_folder(label_folder,"12","./csvs/a_2_features_with_labels_12.csv")

# label_folder = "./dataset/a_4"
# process_images_in_folder(label_folder,"14","./csvs/a_4_features_with_labels_14.csv")

# label_folder = "./dataset/a_8"
# process_images_in_folder(label_folder,"18","./csvs/a_8_features_with_labels_18.csv")

# label_folder = "./dataset/a_16"
# process_images_in_folder(label_folder,"16","./csvs/a_16_features_with_labels_16.csv")

# label_folder = "./dataset/a_32"
# process_images_in_folder(label_folder,"32","./csvs/a_32_features_with_labels_32.csv")

# label_folder = "./dataset/double_flat"
# process_images_in_folder(label_folder,"33","./csvs/double_flat_features_with_labels_33.csv")

# label_folder = "./dataset/double_sharp"
# process_images_in_folder(label_folder,"34","./csvs/double_sharp_features_with_labels_34.csv")

# label_folder = "./dataset/flat"
# process_images_in_folder(label_folder,"35","./csvs/flat_features_with_labels_35.csv")

# label_folder = "./dataset/natural"
# process_images_in_folder(label_folder,"36","./csvs/natural_features_with_labels_36.csv")

# label_folder = "./dataset/sharp"
# process_images_in_folder(label_folder,"37","./csvs/sharp_features_with_labels_37.csv")

# label_folder = "./dataset/digits_dataset/1"
# process_images_in_folder(label_folder,"1","./csvs/1_features_with_labels.csv")


# label_folder = "./dataset/b_2"
# process_images_in_folder(label_folder,"22","./csvs/b_2_features_with_labels_22.csv")
# label_folder = "./dataset/b_4"
# process_images_in_folder(label_folder,"24","./csvs/b_4_features_with_labels_24.csv")
# label_folder = "./dataset/b_8"
# process_images_in_folder(label_folder,"28","./csvs/b_8_features_with_labels_28.csv")
label_folder = "./dataset/b_16"
process_images_in_folder(label_folder,"26","./csvs/b_16_features_with_labels_26.csv")
# label_folder = "./dataset/b_32"
# process_images_in_folder(label_folder,"232","./csvs/b_32_features_with_labels_232.csv")


In [ ]:

# Step 1: Load All CSV Files from a Folder
def load_csvs_from_folder(folder_path):
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = [pd.read_csv(csv_file) for csv_file in csv_files]
    return  pd.concat(dataframes, ignore_index=True)

# Step 2: Balance Each Dataset to Target Size
# def balance_to_avg(dataframes, target_size=500):
#     balanced_dataframes = []
#     for df in dataframes:
#         if len(df) < target_size:
#             # Augment smaller datasets by resampling
#             augmented_df = resample(
#                 df, 
#                 replace=True, 
#                 n_samples=target_size, 
#                 random_state=42
#             )
#             balanced_dataframes.append(augmented_df)
#         else:
#             # Downsample larger datasets
#             downsampled_df = df.sample(n=target_size, random_state=42)
#             balanced_dataframes.append(downsampled_df)
#     return pd.concat(balanced_dataframes)

# Step 3: Train and Evaluate Classifiers
def train_classifiers(data):
    # Separate features and labels
    X = data.iloc[:, :-1].values  # All columns except the last one
    y = data.iloc[:, -1].values   # Last column as target
    
    # Convert labels to numeric if necessary
    if y.dtype == 'object':
        y = y.astype(int)
    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("X_train")
    print(X_train)
    # Define classifiers
    # svc = SVC(kernel='linear', probability=True, random_state=42)
    svc = SVC(kernel='poly',degree=2,C=0.1,gamma=0.1,probability=True,random_state=42,class_weight='balanced') 
    rf = RandomForestClassifier(random_state=42)
    log_reg = LogisticRegression(random_state=42, max_iter=1000)  # Added Logistic Regression

    # # Voting classifier
    # voting_clf = VotingClassifier(
    #     estimators=[('SVM', svc), ('RF', rf)],
    #     voting='soft'
    # )

    # Train classifiers
    # voting_clf.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    rf.fit(X_train, y_train)
    log_reg.fit(X_train, y_train)
    
    # Evaluate classifiers
    # voting_preds = voting_clf.predict(X_test)
    svc_preds = svc.predict(X_test)
    rf_preds = rf.predict(X_test)
    log_reg_preds = log_reg.predict(X_test)

    # Print accuracies
    # print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))
    print("SVM Accuracy:", accuracy_score(y_test, svc_preds))
    print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
    print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_preds))

    return svc,rf,log_reg,scaler

# Specify the folder containing your CSV files
folder_path = "./csvs"  # Replace with your folder path if different

# Load, balance, and combine the datasets
dataframes = load_csvs_from_folder(folder_path)
# balanced_data = balance_to_avg(dataframes, target_size=500)

# Train and evaluate classifiers
svc,rf,log_reg,scaler=train_classifiers(dataframes)


In [ ]:
# 2. Define classifiers
clf1 = LinearDiscriminantAnalysis()
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4= SVC(kernel='poly',degree=2,C=0.1,gamma=0.1) 

# Specify the folder containing your CSV files
folder_path = "./csvs"  # Replace with your folder path if different

# Load, balance, and combine the datasets
dataframes = load_csvs_from_folder(folder_path)
# balanced_data = balance_to_avg(dataframes, target_size=500)
classifiers = [('LDA', clf1), ('RF', clf2), ('GNB', clf3),('SVC', clf4)]
X = dataframes.iloc[:, :-1].values  # All columns except the last one
y = dataframes.iloc[:, -1].values   # Last column as target

# Convert labels to numeric if necessary
if y.dtype == 'object':
    y = y.astype(int)
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 3. Train and evaluate each classifier individually using cross-validation
for name, clf in classifiers:
    scores = cross_val_score(clf, X, y, cv=5)  
    print(f"{name} Accuracy: {np.mean(scores):.4f}")

# 4. Hard voting classifier
eclf1 = VotingClassifier(estimators=classifiers, voting='hard')

# 5. Evaluate the hard voting classifier using cross-validation
scores = cross_val_score(eclf1, X, y, cv=5)  
print(f"Hard Voting Classifier Accuracy: {np.mean(scores):.4f}")

In [ ]:

#Split data into test and validation sets
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# Step 2: Define the parameter grid for GridSearchCV
param_grid = {
    'degree': [1,2],  # Test different polynomial degrees
    'C': [0.01,0.1, 1],  # Test different values for regularization parameter C
    'gamma': [0.01,0.1, 1]  # Test different gamma values
}

# Step 3: Create the SVM model
svm_model = SVC(kernel='poly')

# Step 4: Use GridSearchCV for parameter tuning
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_val, y_val)

# Step 5: Get the best parameters and retrain the model
best_params = grid_search.best_params_
best_svm_model = SVC(kernel='poly', **best_params)
best_svm_model.fit(X_train, y_train)  # Retrain on the entire X_train

# Step 6: Evaluate the final model on the test set
test_accuracy = best_svm_model.score(X_test, y_test)

print("Best Parameters:", best_params)
print("Test Accuracy:",test_accuracy)

In [ ]:

# Save models and scaler to .pkl files
# with open('./models/voting_clf.pkl', 'wb') as f:
#     pickle.dump(voting_clf, f)

with open('./modelsNew/svc.pkl', 'wb') as f:
    pickle.dump(svc, f)

with open('./modelsNew/rf.pkl', 'wb') as f:
    pickle.dump(rf, f)

with open('./modelsNew/log_reg.pkl', 'wb') as f:
    pickle.dump(log_reg, f)

with open('./modelsNew/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Models and scaler saved successfully using pickle!")


In [ ]:
# Load models and scaler from .pkl files
# with open('./models/voting_clf.pkl', 'rb') as f:
#     voting_clf = pickle.load(f)

with open('./modelsNew/svc.pkl', 'rb') as f:
    svc = pickle.load(f)

with open('./modelsNew/rf.pkl', 'rb') as f:
    rf = pickle.load(f)

with open('./modelsNew/log_reg.pkl', 'rb') as f:
    log_reg = pickle.load(f)

with open('./modelsNew/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

print("Models and scaler loaded successfully using pickle!")


In [ ]:
# Test on a new input image
def predict_image(image_path, svc, rf, log_reg, scaler):
     # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Use grayscale for HOG
    if img is None:
        print(f"Failed to load image: {image_path}")

    # # Convert the grayscale image to binary using thresholding
    # _, binary_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    # print(binary_img)
    inverted_img=img

    # # Check if the background (top-left pixel) is dark
    background_pixel = img[0, 0]
    if background_pixel <128:  # Background is dark
        print(f"Inverting image")
        inverted_img = 255-img

    
    show_images([inverted_img])
    resized_image=resize_image(inverted_img)
    
    # Preprocess the image and extract features
    features = extract_hog_features(resized_image)

    # Standardize the features using the same scaler as during training
    features = scaler.transform([features])

    # Predict using each classifier
    # voting_prediction = voting_clf.predict(features)[0]
    svc_prediction = svc.predict(features)[0]
    rf_prediction = rf.predict(features)[0]
    log_reg_prediction = log_reg.predict(features)[0]

    return {
        # "Voting Classifier": voting_prediction,
        "SVM": svc_prediction,
        "Random Forest": rf_prediction,
        "Logisitc Regression": log_reg_prediction,
    }

for i in range(8,65):
    image_path =  f"./processing_output/{i}.png"
    predictions = predict_image(image_path,  svc, rf,log_reg, scaler)

    print("\nPredictions for the input image:")
    for classifier, prediction in predictions.items():
        print(f"{classifier}: {prediction}")


In [8]:
# Define a mapping for predictions
label_mapping = {
    1: "digit 1",
    2: "digit 2",
    3: "digit 3",
    4: "digit 4",
    5: "digit 5",
    6: "digit 6",
    7: "digit 7",
    8: "digit 8",
    9: "digit 9",
    11: "a_1",
    12: "a_2",
    14: "a_4",
    18: "a_8",
    16: "a_16",
    32: "a_32",
    33: "double flat",
    34: "double sharp",
    35: "flat",
    36: "natural",
    37: "sharp",
    22: "b_2",
    24: "b_4",
    28: "b_8",
    26: "b_16",
    232: "b_32",
}
